In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Midterm 2
</center>
</h1>
<div class=h1_cell>
<p>
There are 2 questions on the midterm. I will give partial credit for good progress toward a solution.
<p>
We will use the Titanic table for this midterm but it is only needed for question 2.
<p>
Exam hint: I would look at the 2 main questions first and choose the one that is most straightforward for you. I will give them roughly equal weight with adjustments depending on how the entire class does on each.
</div>

In [2]:
import pandas as pd
import os

home_path =  os.path.expanduser('~')

file_path = '/Desktop/CIS399/'

titanic_table = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR2XbLnUuYlxaFMdS8_bBX3iKUIDEii6Lg5Rxesf-Oh8a6z8-vAN6UDGejaOrBg5130h4O_dLkecKWQ/pub?output=csv')

pd.__version__  # should see 0.20.3 or higher

u'0.20.3'

In [3]:
pd.set_option('display.max_columns', None)

In [5]:
os.chdir(home_path + file_path + 'datascience_1/')
!git pull

Already up-to-date.


In [6]:
import sys
sys.path.append(home_path + '/Dropbox/cis399_ds1_f17/week_libraries/datascience_1')

In [7]:
from week6 import *

%who function

accuracy	 build_pred	 build_tree_iter	 caser	 compute_prediction	 compute_training	 f1	 find_best_splitter	 forest_builder	 
generate_table	 gig	 gini	 informedness	 k_fold	 predictor_case	 probabilities	 seeder	 tree_predictor	 
vote_taker	 


In [8]:
titanic_table.head(1)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,no_age,filled_age,emb_C,emb_Q,emb_S,emb_nan,age_bin,age_Child,age_Adult,age_Senior,sex_female,sex_male,ok_child,pclass_1,pclass_2,pclass_3,pclass_nan
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,0,22.0,0,0,1,0,Child,1,0,0,0,1,0,0,0,1,0


<h1>
Question 1: a better slection of test rows for K-Fold
</h1>
<div class=h1_cell>
<p>
I'd like you to build a couple functions that can help K-Fold select more than 1 slice for testing.
</div>

<h2>
Step 1: Take 2 slices for every test set instead of 1
</h2>
<div class=h1_cell>
<p>
In K-Fold we take 1 test slice from the K slices on every cycle. The other K-1 slices are the training set.
<p>
I think 1 slice is too few. I want you to take 2 slices for testing from the K slices, leaving K-2 slices for the training set. I am going to simplify the problem. What I want is a function that takes one argument: K (the total number of slices). It returns a list of pairs, where each pair is a list or tuple of 2 *non-equal* indices taken from range(K). The catch is that I want all possible combinations of indices. Harkening back to discrete math, this ia a combinatoric problem of K take 2 (or N take 2).
<p>
You can check your answer against this oracle - it will tell you the total number of unique pairs you should have:
<p><br>
```
import scipy.misc
scipy.misc.comb(K, 2, exact=True)
```
<p>
As you might guess, I am banning you from using anything other than libraries we have used in prior modules. This includes numpy. I want to see you generate your own solution. Ask me if you are not sure if a library you want to use is ok.
<p>
You do not need to bring in any K-Fold functions. Your sole focus is on the function I want. I assume that once you have it, you could (with more time), modify kfold to use it.
</div>

In [164]:
import scipy.misc  # you can use scipy.misc.comb in your function if you wish

K = 5 
print(scipy.misc.comb(5, 2, exact=True))


combs = lambda l: [(a, b) for i, a in enumerate(l) for b in l[i+1:]] #from here: https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements


def k_take_2(k):
    data = []
    for it in range(0, k):
        data.append(it)
    return combs(data)



10


In [165]:
k_take_2(5)  #should see 10 - order does not matter

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4)]

<h2>
Step 2: Solve the problem for K take m
</h2>
<div class=h1_cell>
<p>
Write a new function that takes 2 arguments, K and m. So the problem is now to generate all combinations of length m. Step 1 used 2 for m. But now m can be any value between 1 and K.
<p>
I would not try this problem until you have solved step 1 above *and* question 2 below. Come back to it last.
<p>
I am not sure how many points I will give for it. I suspect not many because it is kind of hard.
<p>
You can still use the scipy.misc.comb method to tell you how many combos you should generate for spefic values of K and m.
</div>

In [10]:
import math
import random

def k_take_m(k,m):


In [11]:
k_take_m(5,2)  #I print out the total combinations within my function. Optional for you.

10


[[1, 3],
 [0, 4],
 [0, 2],
 [1, 2],
 [2, 3],
 [3, 4],
 [0, 3],
 [1, 4],
 [2, 4],
 [0, 1]]

In [12]:
k_take_m(5,3)

10


[[2, 3, 4],
 [0, 2, 4],
 [0, 1, 4],
 [0, 3, 4],
 [1, 3, 4],
 [0, 1, 2],
 [1, 2, 4],
 [0, 2, 3],
 [1, 2, 3],
 [0, 1, 3]]

<h1>
Question 2: explore tree behavior
</h1>
<div class=h1_cell>
<p>
For prediction purposes, we have been treating a forest as a single predictor. It does call on its trees to get a prediction, but we don't see that. All we see is the final prediction. I'd like to dig deeper into individual tree behavior. I'll ask you to complete a set of steps to do this.
</div>

<h2>
Step 1. Build a matrix of tree predictions
</h2>
<div class=h1_cell>
<p>
The matrix will be a list of lists. Let's say the forest we are working with has 11 trees. Then the first list will be the predictions of the 11 trees for row 0 of the titanic table, i.e., it will be a list of 11 binary values. The next list will be the 11 predictions for row 1 of the titanic table. Given that we have 891 rows in the titanic table, we will have 891 lists in our outer list.
<p>
Let's say I store my matrix in `all_trees`. Then `all_trees[i][j]` will represent the list of votes for the ith row in titanic table. The jth item in that list will be the vote for tree j. Here is a screenshot of what my all_trees looks like for first several rows. Your answer may differ given randomness involved.
<p>
<img src='https://www.dropbox.com/s/vyxksktwfhtsxlr/Screenshot%202018-02-24%2017.25.44.png?raw=1'>
<p>
Hint 1: you should not be adding any new columns to the Titanic table.
<p>
Hint 2: instead of producing a single vote from a forest, produce a list of votes.
</div>

In [45]:
splitter_columns = [
 'emb_C',
 'emb_Q',
 'emb_S',
 'emb_nan',
 'age_Child',
 'age_Adult',
 'age_Senior',
 'no_age',
 'ok_child',
 'sex_female',
 'pclass_1',
 'pclass_2',
 'pclass_3',
 'pclass_nan'
]

<h2>
Here are a few forests to test on.
</h2>
<div class=h1_cell>
<p>
</div>

In [15]:
forest2 = forest_builder(titanic_table, splitter_columns, 'Survived', hypers={})
len(forest2)

5

In [82]:
forest3 = forest_builder(titanic_table, splitter_columns, 'Survived', hypers={'total-trees':11})
len(forest3)

def vote_taker_new(row, forest):
    newlist = []
    for tree in forest:
        prediction = tree_predictor(row, tree)
        newlist.append(prediction)
    return newlist

row0 = titanic_table.loc[2]
votes = vote_taker(row0, forest3)

all_votes = titanic_table.apply(lambda row: vote_taker_new(row, forest3), axis=1)

list(all_votes) ## DIFFERENT ORDER THAN YOUR ANSWER, SAME NUMBER OF 0's and 1's as yours
#print(votes.keys(), votes.values())

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
 [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 1, 0,

In [84]:
#Compute the matrix and store in all_votes
all_votes = list(all_votes)

all_votes
    

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
 [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 1, 0,

<h2>
Step 2: find the rows where all trees have the same answer
</h2>
<div class=h1_cell>
<p>
Right or wrong, they are unanimous. Place your answer in `unan_rows`.
<p>
If you cannot find the answer for step 1, here are some test rows you can use for step 2 and step 3.
</div>

In [129]:
def all_same(items): ## from https://stackoverflow.com/questions/3787908/python-determine-if-all-items-of-a-list-are-the-same-item
    return all(x == items[0] for x in items)


row0 = titanic_table.loc[873] ## using this to check my output 
votes2 = vote_taker(row0, forest3)
print(votes2)

    
def unanimous_rows(all_votes):
    samelist = []
    index = 0 
    for row in all_votes:
        tup = (all_same(row), index)
        samelist.append(tup)
        index += 1 

    unan_rows = []
    for elem in samelist:
        #print(elem[0]) # True or false if it's unanimous 
        if elem[0] == True:
            unan_rows.append(elem)
    return(unan_rows)



unanimous_rows(all_votes)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[(True, 0),
 (True, 4),
 (True, 5),
 (True, 12),
 (True, 13),
 (True, 17),
 (True, 20),
 (True, 21),
 (True, 29),
 (True, 33),
 (True, 37),
 (True, 45),
 (True, 46),
 (True, 51),
 (True, 59),
 (True, 67),
 (True, 69),
 (True, 70),
 (True, 74),
 (True, 75),
 (True, 76),
 (True, 77),
 (True, 80),
 (True, 81),
 (True, 86),
 (True, 87),
 (True, 89),
 (True, 90),
 (True, 91),
 (True, 93),
 (True, 94),
 (True, 95),
 (True, 99),
 (True, 101),
 (True, 103),
 (True, 104),
 (True, 105),
 (True, 107),
 (True, 108),
 (True, 112),
 (True, 115),
 (True, 116),
 (True, 117),
 (True, 121),
 (True, 126),
 (True, 127),
 (True, 129),
 (True, 131),
 (True, 138),
 (True, 143),
 (True, 146),
 (True, 148),
 (True, 149),
 (True, 150),
 (True, 152),
 (True, 153),
 (True, 154),
 (True, 157),
 (True, 158),
 (True, 159),
 (True, 160),
 (True, 162),
 (True, 163),
 (True, 165),
 (True, 169),
 (True, 173),
 (True, 175),
 (True, 176),
 (True, 178),
 (True, 179),
 (True, 182),
 (True, 188),
 (True, 189),
 (True, 196),


In [127]:
unan_rows = unanimous_rows(all_votes)

In [128]:
len(unan_rows) ## I'm getting more than you are and I don't know why, but I tested my functions and I don't see examples of them returning wrong values
## PLEASE PLEASE PLEASE explain to me what I am doing wrong and why my len is more than yours. 

364

<h2>
Step 3: find the trees that give the same answer for all rows
</h2>
<div class=h1_cell>
<p>
Which trees, if any, always produce the same prediction. Place your answer in `constant_trees`. The values in the list are tree indices.
</div>

In [163]:
def checkEqual3(lst):
    return lst[1:] == lst[:-1]


def never_change(all_votes):
    j = 0 
    list_of_lists = [] # new list of each trees votes 
    for i in range(len(all_votes[0])): # for i in range 11
        lst2 = [item[i] for item in all_votes]
        list_of_lists.append(lst2)

        
##UNFINISHED, 

##TODO: list of lists[0] holds the 0th tree's prediction values
## I'd sort each of the lists within list of lists and check if each list is equal to the each other list 
##DIDN'T HAVE ENOUGH TIME TO GET TO THAT PART 
print(list_of_lists)

[[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,

In [26]:
nc = never_change(all_votes)
len(nc)  #looks like we have no constant tree voters

0

In [ ]:
#test on 2 dummy lists

never_change([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
              [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

#should get a list of [0,5]